## Importing libraries ##

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to C:\Users\SANKET
[nltk_data]     NICHAT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SANKET
[nltk_data]     NICHAT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SANKET
[nltk_data]     NICHAT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
df = pd.read_excel('drugsCom_raw.xlsx')
df

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,2012-05-20,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,2010-04-27,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,2009-12-14,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,2015-11-03,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,2016-11-27,37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,2015-05-31,125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,2011-11-01,34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,2014-03-15,35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,2015-09-19,79


In [24]:
New_df = df[df['condition'].isin(['Depression', 'High Blood Pressure', 'Diabetes, Type 2'])]
New_df

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,2017-03-09,54
31,96233,Sertraline,Depression,"""1 week on Zoloft for anxiety and mood swings....",8,2011-05-07,3
44,121333,Venlafaxine,Depression,"""my gp started me on Venlafaxine yesterday to ...",4,2016-04-27,3
50,156544,Dulaglutide,"Diabetes, Type 2","""Hey Guys, It&#039;s been 4 months since my l...",10,2017-10-24,24
67,131909,Effexor XR,Depression,"""This medicine saved my life. I was at my wits...",10,2013-06-20,166
...,...,...,...,...,...,...,...
161251,198130,Metoprolol,High Blood Pressure,"""It is fourth blood pressure pill for me. It f...",4,2009-07-15,39
161258,34443,Bystolic,High Blood Pressure,"""While on Bystolic my feet and arms were numb....",1,2009-07-18,25
161278,86533,Invokana,"Diabetes, Type 2","""I just got diagnosed with type 2. My doctor p...",9,2015-10-23,47
161286,93069,Vortioxetine,Depression,"""This is the third med I&#039;ve tried for anx...",2,2016-07-17,33


In [7]:
New_df.isnull().sum()

Unnamed: 0     0
drugName       0
condition      0
review         0
rating         0
date           0
usefulCount    0
dtype: int64

In [6]:
New_df['review']

11        "I have taken anti-depressants for years, with...
31        "1 week on Zoloft for anxiety and mood swings....
44        "my gp started me on Venlafaxine yesterday to ...
50        "Hey Guys,  It&#039;s been 4 months since my l...
67        "This medicine saved my life. I was at my wits...
                                ...                        
161251    "It is fourth blood pressure pill for me. It f...
161258    "While on Bystolic my feet and arms were numb....
161278    "I just got diagnosed with type 2. My doctor p...
161286    "This is the third med I&#039;ve tried for anx...
161290    "I have only been on Tekturna for 9 days. The ...
Name: review, Length: 13944, dtype: object

In [25]:

New_df = New_df.drop(columns=['Unnamed: 0', 'drugName', 'rating', 'date', 'usefulCount'])
New_df


,condition,review
11,Depression,"""I have taken anti-depressants for years, with..."
31,Depression,"""1 week on Zoloft for anxiety and mood swings...."
44,Depression,"""my gp started me on Venlafaxine yesterday to ..."
50,"Diabetes, Type 2","""Hey Guys, It&#039;s been 4 months since my l..."
67,Depression,"""This medicine saved my life. I was at my wits..."
...,...,...
161251,High Blood Pressure,"""It is fourth blood pressure pill for me. It f..."
161258,High Blood Pressure,"""While on Bystolic my feet and arms were numb...."
161278,"Diabetes, Type 2","""I just got diagnosed with type 2. My doctor p..."
161286,Depression,"""This is the third med I&#039;ve tried for anx..."


In [29]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

New_df['cleaned_review'] = New_df['review'].apply(clean_text)


In [31]:
stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

New_df['cleaned_review'] = New_df['cleaned_review'].apply(remove_stop_words)


In [33]:
New_df['tokenized_review'] = New_df['cleaned_review'].apply(word_tokenize)


In [35]:
lemmatizer = WordNetLemmatizer()

def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

New_df['lemmatized_review'] = New_df['tokenized_review'].apply(lemmatize_words)


In [37]:
# Initialize the vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features for efficiency

# Fit and transform the cleaned review data
X = tfidf.fit_transform(New_df['cleaned_review'])

# Convert to an array if needed
X = X.toarray()


In [39]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
encoder = LabelEncoder()

# Encode the target (condition) column
y = encoder.fit_transform(New_df['condition'])

# To decode labels later (for interpretation)
condition_mapping = dict(zip(encoder.classes_, range(len(encoder.classes_))))
print(condition_mapping)


{'Depression': 0, 'Diabetes, Type 2': 1, 'High Blood Pressure': 2}


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initialize the Logistic Regression model
log_reg_model = LogisticRegression(max_iter=1000)

# Train the model
log_reg_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [45]:
# Predict on test data
y_pred = log_reg_model.predict(X_test)


In [47]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9530297597705271

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      1800
           1       0.98      0.92      0.95       501
           2       0.95      0.84      0.89       488

    accuracy                           0.95      2789
   macro avg       0.96      0.92      0.94      2789
weighted avg       0.95      0.95      0.95      2789



In [49]:
new_review = ["This medicine saved my life. I feel so much better now."]
new_review_vectorized = tfidf.transform(new_review)  # Vectorize the new review
prediction = log_reg_model.predict(new_review_vectorized)
predicted_condition = encoder.inverse_transform(prediction)
print("Predicted Condition:", predicted_condition[0])


Predicted Condition: Depression
